# Basic RAG
*Credits: example is based on tutorial by Mistral.ai team https://github.com/mistralai/cookbook*
![](https://github.com/mistralai/cookbook/blob/main/images/rag.png?raw=1)

Retrieval-augmented generation (RAG) is an AI framework that synergizes the capabilities of LLMs and information retrieval systems. It’s useful to answer questions or generate content leveraging external knowledge. There are two main steps in RAG: 1) retrieval: retrieve relevant information from a knowledge base with text embeddings stored in a vector store; 2) generation: insert the relevant information to the prompt for the LLM to generate information. In this guide, we will walk through a very basic example of RAG with four implementations:

- RAG from scratch with Mistral
- RAG with Mistral and LangChain
- RAG with Mistral and LlamaIndex
- RAG with Mistral and Haystack

## RAG from scratch

This section aims to guide you through the process of building a basic RAG from scratch. We have two goals: firstly, to offer users a comprehensive understanding of the internal workings of RAG and demystify the underlying mechanisms; secondly, to empower you with the essential foundations needed to build an RAG using the minimum required dependencies.


### Import needed packages
The first step is to install the needed packages `mistralai` and `faiss-cpu` and import the needed packages:



In [2]:
! pip install faiss-cpu==1.7.4 mistralai

In [2]:
from mistralai import Mistral
import requests
import numpy as np
import faiss
import os
from getpass import getpass

api_key = getpass("Type your API Key")
client = Mistral(api_key=api_key)

Type your API Key··········


## Get data and split document into chunks

In a RAG system, it is crucial to split the document into smaller chunks so that it’s more effective to identify and retrieve the most relevant information in the retrieval process later. In this example, we simply split our text by character, combine 2048 characters into each chunk, and we get 37 chunks.

In [50]:
import nltk

In [51]:
!wget https://raw.githubusercontent.com/neychev/small_DL_repo/refs/heads/master/datasets/onegin.txt

--2024-10-02 18:31:33--  https://raw.githubusercontent.com/neychev/small_DL_repo/refs/heads/master/datasets/onegin.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 262521 (256K) [text/plain]
Saving to: ‘onegin.txt’

onegin.txt          100%[===================>] 256.37K  --.-KB/s    in 0.04s   

2024-10-02 18:31:33 (6.77 MB/s) - ‘onegin.txt’ saved [262521/262521]



In [54]:
with open("onegin.txt", "r") as iofile:
    text_raw = iofile.readlines()

In [56]:
text = "\n".join([x.strip() for x in text_raw if len(x.strip()) > 0])

In [57]:
print(text[:200])

I
«Мой дядя самых честных правил,
Когда не в шутку занемог,
Он уважать себя заставил
И лучше выдумать не мог.
Его пример другим наука;
Но, боже мой, какая скука
С больным сидеть и день и ночь,
Не отхо


In [64]:
tokenized_sentences = nltk.sent_tokenize(text, language="russian")

In [65]:
max_length = 1000

In [66]:
chunks = []
current_chunk = ""
for sentence in tokenized_sentences:
    if len(current_chunk) + len(sentence) + 1 <= max_length:
        current_chunk += " " + sentence
    else:
        if current_chunk:
            chunks.append(current_chunk.strip())
        current_chunk = sentence
if current_chunk:
    chunks.append(current_chunk.strip())

In [68]:
print(chunks[0])

I
«Мой дядя самых честных правил,
Когда не в шутку занемог,
Он уважать себя заставил
И лучше выдумать не мог. Его пример другим наука;
Но, боже мой, какая скука
С больным сидеть и день и ночь,
Не отходя ни шагу прочь! Какое низкое коварство
Полуживого забавлять,
Ему подушки поправлять,
Печально подносить лекарство,
Вздыхать и думать про себя:
Когда же черт возьмет тебя!»
II
Так думал молодой повеса,
Летя в пыли на почтовых,
Всевышней волею Зевеса
Наследник всех своих родных. —
Друзья Людмилы и Руслана! С героем моего романа
Без предисловий, сей же час
Позвольте познакомить вас:
Онегин, добрый мой приятель,
Родился на брегах Невы,
Где, может быть, родились вы
Или блистали, мой читатель;
Там некогда гулял и я:
Но вреден север для меня
III
Служив отлично-благородно,
Долгами жил его отец,
Давал три бала ежегодно
И промотался наконец. Судьба Евгения хранила:
Сперва Madame за ним ходила,
Потом Monsieur ее сменил;
Ребенок был резов, но мил.


#### Considerations:
- **Chunk size**: Depending on your specific use case, it may be necessary to customize or experiment with different chunk sizes and chunk overlap to achieve optimal performance in RAG. For example, smaller chunks can be more beneficial in retrieval processes, as larger text chunks often contain filler text that can obscure the semantic representation. As such, using smaller text chunks in the retrieval process can enable the RAG system to identify and extract relevant information more effectively and accurately.  However, it’s worth considering the trade-offs that come with using smaller chunks, such as increasing processing time and computational resources.
- **How to split**: While the simplest method is to split the text by character, there are other options depending on the use case and document structure. For example, to avoid exceeding token limits in API calls, it may be necessary to split the text by tokens. To maintain the cohesiveness of the chunks, it can be useful to split the text by sentences, paragraphs, or HTML headers. If working with code, it’s often recommended to split by meaningful code chunks for example using an Abstract Syntax Tree (AST) parser.


### Create embeddings for each text chunk
For each text chunk, we then need to create text embeddings, which are numeric representations of the text in the vector space. Words with similar meanings are expected to be in closer proximity or have a shorter distance in the vector space.
To create an embedding, use Mistral’s embeddings API endpoint and the embedding model `mistral-embed`. We create a `get_text_embedding` to get the embedding from a single text chunk and then we use list comprehension to get text embeddings for all text chunks.


In [70]:
def get_text_embedding(input):
    embeddings_batch_response = client.embeddings.create(
        model="mistral-embed", inputs=input
    )
    return embeddings_batch_response.data[0].embedding

In [71]:
import time

In [72]:
embed_list = []

In [1]:
for chunk in tqdma(chunks[len(embed_list) :]):
    embed_list.append(get_text_embedding(chunk))
    time.sleep(1.5)

In [74]:
text_embeddings = np.array(embed_list)

In [75]:
text_embeddings.shape

(25, 1024)

### Load into a vector database
Once we get the text embeddings, a common practice is to store them in a vector database for efficient processing and retrieval. There are several vector database to choose from. In our simple example, we are using an open-source vector database Faiss, which allows for efficient similarity search.  

With Faiss, we instantiate an instance of the Index class, which defines the indexing structure of the vector database. We then add the text embeddings to this indexing structure.


In [76]:
d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

#### Considerations:
- **Vector database**: When selecting a vector database, there are several factors to consider including speed, scalability, cloud management, advanced filtering, and open-source vs. closed-source.

### Create embeddings for a question
Whenever users ask a question, we also need to create embeddings for this question using the same embedding models as before.


In [77]:
question = "Что ел Евгений в ресторане?"
question_embeddings = np.array([get_text_embedding(question)])
question_embeddings.shape

(1, 1024)

In [78]:
question_embeddings[0].dot(text_embeddings.T)

array([0.78836663, 0.79888681, 0.76348379, 0.790934  , 0.78910528,
       0.81562656, 0.7685063 , 0.78843981, 0.78624877, 0.78812345,
       0.75959841, 0.77016986, 0.78904237, 0.79210282, 0.77717381,
       0.81304187, 0.75823373, 0.80236302, 0.77026705, 0.80386935,
       0.7472646 , 0.77184632, 0.74869756, 0.81027778, 0.79531215])

#### Considerations:
- Hypothetical Document Embeddings (HyDE): In some cases, the user’s question might not be the most relevant query to use for identifying the relevant context. Instead, it maybe more effective to generate a hypothetical answer or a hypothetical document based on the user’s query and use the embeddings of the generated text to retrieve similar text chunks.

### Retrieve similar chunks from the vector database
We can perform a search on the vector database with `index.search`, which takes two arguments: the first is the vector of the question embeddings, and the second is the number of similar vectors to retrieve. This function returns the distances and the indices of the most similar vectors to the question vector in the vector database. Then based on the returned indices, we can retrieve the actual relevant text chunks that correspond to those indices.


In [79]:
question_embeddings[0].dot(text_embeddings.T).argsort()[::-1]

array([ 5, 15, 23, 19, 17,  1, 24, 13,  3,  4, 12,  7,  0,  9,  8, 14, 21,
       18, 11,  6,  2, 10, 16, 22, 20])

In [80]:
D, I = index.search(question_embeddings, k=2)
print(I)

[[ 5 15]]


In [81]:
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
print(retrieved_chunk)

['Покамест в утреннем уборе,\nНадев широкий боливар\nОнегин едет на бульвар,\nИ там гуляет на просторе,\nПока недремлющий брегет\nНе прозвонит ему обед. XVI\nУж темно: в санки он садится. «Пади, пади!» – раздался крик;\nМорозной пылью серебрится\nЕго бобровый воротник. К Talon помчался: он уверен,\nЧто там уж ждет его Каверин. Вошел: и пробка в потолок,\nВина кометы брызнул ток;\nПред ним roast-beef окровавленный\nИ трюфли, роскошь юных лет,\nФранцузской кухни лучший цвет,\nИ Страсбурга пирог нетленный\nМеж сыром лимбургским живым\nИ ананасом золотым. XVII\nЕще бокалов жажда просит\nЗалить горячий жир котлет,\nНо звон брегета им доносит,\nЧто новый начался балет. Театра злой законодатель,\nНепостоянный обожатель\nОчаровательных актрис,\nПочетный гражданин кулис,\nОнегин полетел к театру,\nГде каждый, вольностью дыша,\nГотов охлопать entrechat,\nОбшикать Федру, Клеопатру,\nМоину вызвать (для того,\nЧтоб только слышали его). XVIII\nВолшебный край!', 'Всех прежде вас оставил он;\nИ правда

#### Considerations:
- **Retrieval methods**: There are a lot different retrieval strategies. In our example, we are showing a simple similarity search with embeddings. Sometimes when there is metadata available for the data, it’s better to filter the data based on the metadata first before performing similarity search. There are also other statistical retrieval methods like TF-IDF and BM25 that use frequency and distribution of terms in the document to identify relevant text chunks.
- **Retrieved document**: Do we always retrieve individual text chunk as it is? Not always.
    - Sometimes, we would like to include more context around the actual retrieved text chunk. We call the actual retrieve text chunk “child chunk” and our goal is to retrieve a larger “parent chunk” that the “child chunk” belongs to.
    - On occasion, we might also want to provide weights to our retrieve documents. For example, a time-weighted approach would help us retrieve the most recent document.
    - One common issue in the retrieval process is the “lost in the middle” problem where the information in the middle of a long context gets lost. Our models have tried to mitigate this issue. For example, in the passkey task, our models have demonstrated the ability to find a "needle in a haystack" by retrieving a randomly inserted passkey within a long prompt, up to 32k context length. However, it is worth considering experimenting with reordering the document to determine if placing the most relevant chunks at the beginning and end leads to improved results.
  
### Combine context and question in a prompt and generate response

Finally, we can offer the retrieved text chunks as the context information within the prompt. Here is a prompt template where we can include both the retrieved text and user question in the prompt.



In [86]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
You must answer in Russian and your answer should be a bullet-item list.
Query: {question}
Answer:
"""

In [87]:
def run_mistral(user_message, model="mistral-large-latest"):
    messages = [{"role": "user", "content": user_message}]
    chat_response = client.chat.complete(model=model, messages=messages)
    return (chat_response.choices[0].message.content, chat_response)

In [88]:
output, full_response = run_mistral(prompt)

In [89]:
print(output)

- Roast-beef окровавленный
- Трюфли
- Страсбурга пирог нетленный
- Сыр лимбургский живой
- Ананас золотой


#### Considerations:
- Prompting techniques: Most of the prompting techniques can be used in developing a RAG system as well. For example, we can use few-shot learning to guide the model’s answers by providing a few examples. Additionally, we can explicitly instruct the model to format answers in a certain way.


In the next sections, we are going to show you how to do a similar basic RAG with some of the popular RAG frameworks. We will start with LlamaIndex and add other frameworks in the future.
